<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla, Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias, Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Catherine Benavides
- Nombre de alumno 2: Fernanda Saavedra


### **Link de repositorio de GitHub:**  https://github.com/FernandaSD/MDS7202

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

## Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

### Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [2]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de datos presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [4]:
df_retail = pd.read_pickle("/content/drive/MyDrive/online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

**Observación:** Se obtienen datos nulos cuando las personas realizan una sola compra al momento de calcular la periodicidad de las observaciones. Dado esto, se imputaron estos datos faltantes con el valor cero, dado que una persona que realiza una sola compra no presenta periodicidad.

In [11]:
def custom_features(dataframe_in): 
  # Se copia el dataframe y se calcula el gasto de los clientes
  df = dataframe_in.copy()
  df['Gastos'] = df['Quantity'] * df['Price']

  # Se agrupan los datos por transacción
  # Se mantienen solo las columnas que se necesitan para calcular los LRMFP
  # Se suman gastos; se saca total por transacción
  # Se mantiene una fecha y un id de cliente por transacción
  invoice = df.groupby('Invoice').agg({'Customer ID': 'first', 
                                        'Gastos': 'sum',
                                        'InvoiceDate': 'first'})
  invoice = invoice.reset_index()
  
  # Dataframe para guardar las características obtenidas
  dataframe_out = pd.DataFrame()


  # Length 
  # Se calcula la primera vez de visita del cliente y la última 
  min = invoice.groupby('Customer ID')['InvoiceDate'].min()
  max = invoice.groupby('Customer ID')['InvoiceDate'].max()

  # Se restan ambas fechas para sacar el intervalo de días 
  dataframe_out['Length'] = (max - min).apply(lambda x: x.days)


  # Recency
  # Se calcula la fecha final de observación + un día
  end_date = df['InvoiceDate'].max() + datetime.timedelta(days = 1)

  # Diferencia en días entre fecha final y la fecha de visita
  invoice['diff_dates'] = (end_date - invoice['InvoiceDate']).apply(lambda x: x.days)

  # Se saca el promedio de días por cliente
  dataframe_out['Recency'] = invoice.groupby('Customer ID')['diff_dates'].mean()
  

  # Frecuency
  # Se cuentan las transacciones por cliente
  dataframe_out['Frequency'] = invoice.groupby('Customer ID')['Invoice'].count()


  # Monetary
  # Se saca el promedio de gasto por cliente
  mean = invoice.groupby('Customer ID')['Gastos'].mean()

  # Se redondea el promedio
  dataframe_out['Monetary'] = round(mean, 3)


  # Periodicity
  # Se calcula la diferencia en días entre las fechas de transacción por cliente
  invoice = invoice.sort_values(by="InvoiceDate")
  invoice['date_diff'] = invoice.groupby('Customer ID')['InvoiceDate'].diff()
  invoice['date_diff'] = invoice['date_diff'].apply(lambda x: x.days)
  invoice['date_diff'] = invoice['date_diff'].fillna(0) 

  # Se calcula std por cliente y se redondea
  std_days = invoice.groupby('Customer ID')['date_diff'].std().fillna(0)  
  dataframe_out['Periodicity'] = round(std_days, 3)

  return dataframe_out

In [12]:
custom_features(df_retail)

,Length,Recency,Frequency,Monetary,Periodicity
Customer ID,,,,,
12346.0,196,325.181818,11,33.896,35.257
12347.0,37,21.500000,2,661.660,26.163
12348.0,0,74.000000,1,222.160,0.000
12349.0,181,158.000000,3,890.380,88.792
12351.0,0,11.000000,1,300.930,0.000
...,...,...,...,...,...
18283.0,275,163.833333,6,103.228,59.628
18284.0,0,67.000000,1,461.680,0.000
18285.0,0,296.000000,1,427.000,0.000


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si sería posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [13]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
      # Se obtienen el mínimo y máximo de la columna
      self.min = X.min()
      self.max = X.max()
      
      return self

    def transform(self, X):
      # Numerador y denominador de la ecuación
      numerador = X - self.min
      denominador = self.max - self.min

      # Se calcula el nuevo valor
      MinMax = numerador / denominador
      
      return MinMax

    def fit_transform(self, X, y = None):
      # Se obtienen el mínimo y máximo de la columna
      self.min = X.min()
      self.max = X.max()

      # Numerador y denominador de la ecuación
      numerador = X - self.min
      denominador = self.max - self.min

      # Se calcula el nuevo valor
      MinMax = numerador / denominador
      
      return MinMax

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [14]:
# Se aplica FunctionTransformer() a la función de la parte 1.2
custom = FunctionTransformer(custom_features)

# Se crea el column transformer
num_cols = ['Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity']
num_transformer = MinMax()

preprocessor = ColumnTransformer(
    transformers=[('transformer', num_transformer, num_cols)])

# Reducción de dimensionalidad
model = TSNE(n_components = 2, random_state = 20295)

# Configuración pipeline
pipeline = Pipeline(steps=(['features', custom], ['transformer', preprocessor], 
                           ['reduction', model]))

In [15]:
# Se aplica el pipeline en los datos
components = pipeline.fit_transform(df_retail)

# Se pasa a un dataframe
components = pd.DataFrame(components, columns=['Proyección0', 'Proyección1'])

In [16]:
# Gráfico de dispersión
fig = px.scatter(components, x='Proyección0', y='Proyección1', 
                 title='Proyección en 2D')
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [17]:
# Lista para grardar la cantidad de cluster y su inercia 
inertias = []

for k in range(1, 21):
  # Modelo de clustering
  clustering = KMeans(n_clusters = k, random_state = 20295)

  # Configuración pipeline
  pipeline_kmeans = Pipeline(steps=(['features', custom], 
                                    ['transformer', preprocessor], 
                                    ['clusters', clustering]))
  
  # Se aplica el pipeline en los datos
  pipeline_kmeans.fit_transform(df_retail)

  k_inertia = [k, pipeline_kmeans.named_steps['clusters'].inertia_]

  inertias.append(k_inertia)

inertias = pd.DataFrame(inertias, columns=['clusters', 'inertia'])
inertias.head(10)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

,clusters,inertia
0,1,936.367734
1,2,430.375122
2,3,271.543870
3,4,205.420815
4,5,162.614098
5,6,142.220918
6,7,123.757865
7,8,108.046408
8,9,98.172676
9,10,89.516017


In [18]:
# Se grafican las inercias por cantidad de clusters
px.line(inertias, x = 'clusters', y = 'inertia', 
        title = 'Método del Codo con K-Means', height=600)

Observando el gráfico se concluye que no se observa un número claro de clusters a utilizar, considerando el método del codo. Pese a esto, se escogió trabajar con 4 clusters dado que para números superiores no se observa una mejora sustancial en la pendiente de la inercia del modelo.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

**Respuesta:**

In [19]:
# Modelo de clustering
clustering = KMeans(n_clusters = 4, random_state = 20295)

# Configuración pipeline
pipeline_kmeans = Pipeline(steps=(['features', custom], 
                                  ['transformer', preprocessor], 
                                  ['clusters', clustering]))

In [20]:
# Se aplica el pipeline en los datos
pipeline_kmeans.fit_transform(df_retail)

# Se agrega los cluster al dataset
df = custom_features(df_retail)
df['Cluster'] = pipeline_kmeans.named_steps['clusters'].labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [21]:
# Tabla de promedios
df.groupby('Cluster').mean()

,Length,Recency,Frequency,Monetary,Periodicity
Cluster,,,,,
0,193.823586,162.799962,3.517737,364.408839,86.891663
1,21.079941,62.011434,1.888231,368.787281,8.855396
2,17.182025,266.412572,1.604096,352.053309,7.833250
3,318.982709,179.644863,11.128722,418.018307,46.542227


De la tabla de promedios se observa que todos los clusters presentan diferencias en la variable Recency. Con respecto al resto de las variables, los clusters también presentas variaciones sustanciales en sus centros, salvo los clusters 1 y 2, los cuales se parecen más entre sí. Pese a esto, dadas las grandes diferencias observadas en Recency para los clusters 1 y 2, se concluye que 4 clusters es un número adecuado.

Con respecto a 'Length', los cluster 1 y 2 poseen clientes con una baja fidelidad, el cluster 0 con una fidelidad intermedia, y el cluster 3 con una alta fidelidad a comparación del resto de grupos.

A partir de 'Recency' se puede observar que el cluster 1 posee clientes donde sus compras son más recientes que el resto de grupos. En general, los otros 3 cluster tienen clientes con compras antiguas, siendo el cluster 2 donde se ve la mayor antiguedad.

Con respecto a 'Frecuency', los cluster 0, 1 y 2 poseen una baja frecuencia de visitas, siendo los clientes menos fieles, mientras que el cluster 3 presenta la major fidelidad, lo que coincide con lo obtenido para 'Length'.

Para 'Monetary' se tiene que los 4 cluster, en general, aportan cantidades de ingreso similares.

Por último, en 'Periodicity', los cluster 1 y 2 tienen clientes que compran frecuentemente en la tienda, mientras que los dos restantes lo hacen con baja regularidad.

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [22]:
# Se agregan los cluster a las proyecciones anteriores
components['Cluster'] = pipeline_kmeans.named_steps['clusters'].labels_

In [23]:
# Gráfico de dispersión
fig = px.scatter(components, x = 'Proyección0', y = 'Proyección1', 
                 title = 'Proyección en 2D', color = 'Cluster')
fig.show()

De la visualización se comenta que no existen delimitaciones claras entre los clusters. Esto se debe dado que al momento de reducir la dimensión de los datos no se obtienen clusters convexos, por lo cual el algoritmos de k-means no logra hacer una delimitación clara entre ellos. Esto es consistente con el gráfico del codo, ya que no se observa con facilidad un número óptimo de clusters.

# Conclusión

Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>